In [9]:
import sys
sys.path.append('../30_data_tools/')
sys.path.append('../60_gebastel/Musterueberlagerung/pattern_effects/')
sys.path.append('../80_Auswertungen/')

In [10]:
import numpy as np
from PIL import Image, ImageChops, ImageOps
from helper import load_dotenv
from trapezoidal_distortion import distort_trapezoidal
from wave_deform import wave_deform
from blow_up import blow_up, contract
import plotly.express as px
import math
from scipy import ndimage
from load_constants import load_colors

In [3]:
def get_fft( input_img ):
    ft = np.fft.ifftshift(np.array(input_img))
    ft = np.fft.fft2(ft)
    ft = np.fft.fftshift(ft)
    
    return ft

In [4]:
def limit_frequencies( fft, inner_limit=None, outer_limit=None ):
    center = (fft.shape[1] / 2, fft.shape[0] / 2)
    for y in range(fft.shape[0]):
        for x in range(fft.shape[1]):
            r = math.sqrt( abs(center[0] - x) ** 2 + abs(center[1] - y) ** 2 )
            
            if outer_limit is not None and r > outer_limit:
                fft[y,x] = 0
    
            if inner_limit is not None and r < inner_limit:
                fft[y,x] = 0

    return fft

In [5]:
def get_fft_spectrum( grating_img, apply_effect, overlay=True ):
    grating_effect_img = apply_effect( grating_img )
    multiplied_img = ImageChops.multiply( grating_img, grating_effect_img )

    if overlay:
        fft = get_fft( multiplied_img )
    else:
        fft = get_fft( grating_effect_img )
    
    fig = px.imshow(
        ndimage.uniform_filter(
            np.abs(fft),
            size=5
        )[750:1250,750:1250],
        color_continuous_scale=['white','black']
    )
    fig.update_layout(coloraxis_showscale=False)
    fig.update_xaxes(showticklabels=False)
    fig.update_yaxes(showticklabels=False)

    return fig

In [6]:
def get_effect_img( grating_img, apply_effect, margin=0.1):
    grating_background_a = Image.new('L', (round(grating_img.size[0] * (1 + margin * 2)), round(grating_img.size[1] * (1 + margin * 2))), color="white")
    grating_background_a.paste(
        grating_img,
        (round(grating_img.size[0] * margin), round(grating_img.size[1] * margin))
    )
    
    grating_background_b = grating_background_a.copy()
    grating_background_b = apply_effect(grating_background_b)

    return ImageChops.multiply(
        grating_background_a,
        grating_background_b
    )

In [12]:
COLORS = load_colors()

In [7]:
Image.MAX_IMAGE_PIXELS = None

In [8]:
dotenv = load_dotenv()

In [25]:
x = np.arange(-1000, 1001, 1)
X, Y = np.meshgrid(x, x)
wavelength = 20
grating = np.sin(2 * np.pi * X / wavelength) * 0.5 + 0.5
grating[grating < 0] = 0
grating[grating > 1] = 1

grating_2d = np.sin(2 * np.pi * X / wavelength) * 0.5 + 0.77
grating_2d[grating_2d < 0] = 0
grating_2d[grating_2d > 1] = 1

angle = np.pi / 180 * 90
grating_2d_rotated = np.sin(
    2*np.pi*(X*np.cos(angle) + Y*np.sin(angle)) / wavelength
) * 0.5 + 0.77
grating_2d_rotated[grating_2d_rotated < 0] = 0
grating_2d_rotated[grating_2d_rotated > 1] = 1

In [ ]:
grating.mean()

In [ ]:
(grating_2d * grating_2d_rotated).mean()

In [26]:
grating_img = Image.fromarray((((grating) * 255).round()).astype('uint8'))
grating_img_2d = Image.fromarray( ((1 - (grating_2d * grating_2d_rotated)) * 255).round().astype('uint8') )

In [ ]:
get_fft_spectrum(grating_img, lambda img: img, overlay=False).write_image( dotenv['ATTACHMENT_DIR'] / 'moire_kategorien' / 'base_1d_fft.jpg' )
get_fft_spectrum(grating_img_2d, lambda img: img, overlay=False).write_image( dotenv['ATTACHMENT_DIR'] / 'moire_kategorien' / 'base_2d_fft.jpg' )

In [ ]:
grating_img.save( dotenv['ATTACHMENT_DIR'] / 'moire_kategorien' / 'base_1d.jpg' )
grating_img_2d.save( dotenv['ATTACHMENT_DIR'] / 'moire_kategorien' / 'base_2d.jpg' )

# Verzerrung

In [ ]:
distort_strength = 0.2

distort_top = lambda img: distort_trapezoidal( img, [(distort_strength,0),(0,0),(0,0),(distort_strength,0)] )
distort_top_bottom = lambda img: distort_trapezoidal( distort_trapezoidal( img, [(distort_strength,0),(0,0),(0,0),(0,0)] ), [(0,0),(0,0),(distort_strength,0),(0,0)] )
distort_top_left = lambda img: distort_trapezoidal( img, [(distort_strength,0),(0,0),(0,0),(0,0)] )

In [ ]:
get_fft_spectrum( grating_img_2d, lambda img: img ).write_image( dotenv['ATTACHMENT_DIR'] / 'moire_kategorien' / 'distort_none_fft.jpg' )
get_effect_img(
    grating_img_2d,
    lambda img: img,
    margin=0.2
).save( dotenv['ATTACHMENT_DIR'] / 'moire_kategorien' / 'distort_none.jpg' )

get_fft_spectrum( grating_img_2d, distort_top ).write_image( dotenv['ATTACHMENT_DIR'] / 'moire_kategorien' / 'distort_top_fft.jpg' )
get_effect_img(
    grating_img_2d,
    distort_top,
    margin=0.2
).save( dotenv['ATTACHMENT_DIR'] / 'moire_kategorien' / 'distort_top.jpg' )

get_fft_spectrum( grating_img_2d, distort_top_bottom ).write_image( dotenv['ATTACHMENT_DIR'] / 'moire_kategorien' / 'distort_top_bottom_fft.jpg' )
get_effect_img(
    grating_img_2d,
    distort_top_bottom,
    margin=0.2
).save( dotenv['ATTACHMENT_DIR'] / 'moire_kategorien' / 'distort_top_bottom.jpg' )

get_fft_spectrum( grating_img_2d, distort_top_left ).write_image( dotenv['ATTACHMENT_DIR'] / 'moire_kategorien' / 'distort_top_left_fft.jpg' )
get_effect_img(
    grating_img_2d,
    distort_top_left,
    margin=0.2
).save( dotenv['ATTACHMENT_DIR'] / 'moire_kategorien' / 'distort_top_left.jpg' )

In [ ]:
for i in range(1,5):
    distort_by_level = lambda img: distort_trapezoidal( img, [(i / 10,0),(0,0),(0,0),(0,0)] )
    filename = f'distort_top_left_{ i }'
    
    get_fft_spectrum( grating_img_2d, distort_by_level ).write_image( dotenv['ATTACHMENT_DIR'] / 'moire_kategorien' / f'{filename}_fft.jpg' )
    get_effect_img(
        grating_img_2d,
        distort_by_level,
        margin=0.2
    ).save( dotenv['ATTACHMENT_DIR'] / 'moire_kategorien' / f'{filename}.jpg' )

In [ ]:
for i in range(1,5):
    distort_by_level = lambda img: distort_trapezoidal( img, [(i / 10,0),(0,0),(0,0),(0,0)] )
    filename = f'distort_top_left_{ i }'
    
    get_fft_spectrum( grating_img_2d, distort_by_level ).write_image( dotenv['ATTACHMENT_DIR'] / 'moire_kategorien' / f'{filename}_1d_fft.jpg' )
    get_effect_img(
        grating_img_2d,
        distort_by_level,
        margin=0.2
    ).save( dotenv['ATTACHMENT_DIR'] / 'moire_kategorien' / f'{filename}_1d.jpg' )

In [ ]:
get_effect_img(
    grating_img_2d,
    lambda img: distort_trapezoidal( img, [(0.2,0.2),(0,0),(0,0),(0,0)] ),
    margin=0.2
).save( dotenv['ATTACHMENT_DIR'] / 'moire_kategorien' / f'distort_top_2d.jpg' )
get_fft_spectrum( grating_img_2d, lambda img: distort_trapezoidal( img, [(0.2,0.2),(0,0),(0,0),(0,0)] ) ).write_image( dotenv['ATTACHMENT_DIR'] / 'moire_kategorien' / 'distort_top_2d_fft.jpg' )

get_effect_img(
    grating_img_2d,
    lambda img: distort_trapezoidal( img, [(-0.2,-0.2),(0,0),(0,0),(0,0)] ),
    margin=0.2
).save( dotenv['ATTACHMENT_DIR'] / 'moire_kategorien' / f'distort_top_2d_2.jpg' )
get_fft_spectrum( grating_img_2d, lambda img: distort_trapezoidal( img, [(-0.2,-0.2),(0,0),(0,0),(0,0)] ) ).write_image( dotenv['ATTACHMENT_DIR'] / 'moire_kategorien' / 'distort_top_2d_2_fft.jpg' )

get_effect_img(
    grating_img_2d,
    lambda img: distort_trapezoidal( img, [(0.2,0.2),(0,0.2),(0,0),(0,0)] ),
    margin=0.2
).save( dotenv['ATTACHMENT_DIR'] / 'moire_kategorien' / f'distort_top_2d_3.jpg' )
get_fft_spectrum( grating_img_2d, lambda img: distort_trapezoidal( img, [(0.2,0.2),(0,0.2),(0,0),(0,0)] ) ).write_image( dotenv['ATTACHMENT_DIR'] / 'moire_kategorien' / 'distort_top_2d_3_fft.jpg' )

get_effect_img(
    grating_img_2d,
    lambda img: distort_trapezoidal( img, [(0.2,0),(0,0),(0,0),(0,-0.2)] ),
    margin=0.2
).save( dotenv['ATTACHMENT_DIR'] / 'moire_kategorien' / f'distort_top_2d_4.jpg' )
get_fft_spectrum( grating_img_2d, lambda img: distort_trapezoidal( img, [(0.2,0.2),(0,0.2),(0,0),(0,0)] ) ).write_image( dotenv['ATTACHMENT_DIR'] / 'moire_kategorien' / 'distort_top_2d_4_fft.jpg' )

In [21]:
T1 = 1
s2 = 1.2
th1 = 0

t2 = lambda x: x * ((s2 - 1) * T1) / s2 + T1
th2 = lambda x: x * math.degrees(math.atan((s2 - 1) / 1)) / s2 + th1

tm = lambda x: (T1 * t2(x)) / math.sqrt(T1**2 + t2(x) ** 2 - 2 * T1 * t2(x) * math.cos( math.radians(abs(th1 - th2(x)))) )
thm = lambda x: math.atan((t2(x) * math.sin(math.radians(th1) - T1 * math.sin(math.radians(th2(x))))) / (t2(x) * math.cos(math.radians(th1) - T1 * math.cos(math.radians(th2(x))))))

In [22]:
px.line(
    x=[i/100 for i in range(2,101)],
    y=[tm(i/100) for i in range(2,101)],
    labels={
        'y' : 'Periode des Moirémusters'
    },
    color_discrete_sequence=COLORS['COLOR_SEQUENCE'],
    width=1000,
    height=350
).write_image( dotenv['ATTACHMENT_DIR'] / 'moire_kategorien' / f'abhaengigkeit_x_moire_periode.jpg' )

In [23]:
px.line(
    x=[i/100 for i in range(2,101)],
    y=[math.degrees(thm(i/100)) for i in range(2,101)],
    labels={
        'y' : 'Winkel des Moirémusters'
    },
    color_discrete_sequence=COLORS['COLOR_SEQUENCE'],
    width=1000,
    height=350
).write_image( dotenv['ATTACHMENT_DIR'] / 'moire_kategorien' / f'abhaengigkeit_x_moire_winkel.jpg' )

# Rotation

In [31]:
rotate_grating = lambda img: img.rotate(5, fillcolor="white")

In [ ]:
get_fft_spectrum( grating_img_2d, rotate_grating ).write_image( dotenv['ATTACHMENT_DIR'] / 'moire_kategorien' / 'rotate_2d_fft.jpg' )
get_effect_img(
    grating_img_2d,
    rotate_grating,
    margin=0.2
).save( dotenv['ATTACHMENT_DIR'] / 'moire_kategorien' / 'rotate_2d.jpg' )

In [ ]:
get_fft_spectrum( grating_img, rotate_grating ).write_image( dotenv['ATTACHMENT_DIR'] / 'moire_kategorien' / 'rotate_1d_fft.jpg' )
get_effect_img(
    grating_img,
    rotate_grating,
    margin=0.2
).save( dotenv['ATTACHMENT_DIR'] / 'moire_kategorien' / 'rotate_1d.jpg' )

In [ ]:
for i in [2,5,10,25]:
    distort_by_level = lambda img: img.rotate(i, fillcolor="white")
    filename = f'rotate_{ i }'
    
    get_fft_spectrum( grating_img_2d, distort_by_level ).write_image( dotenv['ATTACHMENT_DIR'] / 'moire_kategorien' / f'{filename}_1d_fft.jpg' )
    get_effect_img(
        grating_img_2d,
        distort_by_level,
        margin=0.2
    ).save( dotenv['ATTACHMENT_DIR'] / 'moire_kategorien' / f'{filename}_1d.jpg' )

In [13]:
p = 1
results = []

for i in range(359):
    a = math.radians((i+1))
    p_m = p / (2 * abs(math.sin(a/2)))

    results.append(((i+1), p_m))

In [18]:
px.line(
    x=[r[0] for r in results],
    y=[r[1] for r in results],
    labels={
        'x' : 'Winkelunterschied',
        'y' : 'Periode der Moiréstruktur'
    },
    color_discrete_sequence=COLORS['COLOR_SEQUENCE'],
    width=1000,
    height=350
).write_image( dotenv['ATTACHMENT_DIR'] / 'moire_kategorien' / 'abhaengigkeit_rotation_moire_periode.jpg' )

px.line(
    x=[r[0] for r in results],
    y=[1/r[1] for r in results],
    labels={
        'x' : 'Winkelunterschied',
        'y' : 'Frequenz der Moiréstruktur'
    },
    color_discrete_sequence=COLORS['COLOR_SEQUENCE'],
    width=1000,
    height=350
).write_image( dotenv['ATTACHMENT_DIR'] / 'moire_kategorien' / 'abhaengigkeit_rotation_moire_frequenz.jpg' )

# Skalierung

In [27]:
scale_factor = 0.05
scale_grating = lambda img: img.resize((round(img.size[0] * (1 + scale_factor)), round(img.size[1] * (1 + scale_factor))))

get_fft_spectrum( grating_img_2d, scale_grating ).write_image( dotenv['ATTACHMENT_DIR'] / 'moire_kategorien' / 'scale_2d_fft.jpg' )
get_effect_img(
    grating_img_2d,
    scale_grating,
    margin=0.2
).save( dotenv['ATTACHMENT_DIR'] / 'moire_kategorien' / 'scale_2d.jpg' )

get_fft_spectrum( grating_img, scale_grating ).write_image( dotenv['ATTACHMENT_DIR'] / 'moire_kategorien' / 'scale_1d_fft.jpg' )
get_effect_img(
    grating_img,
    scale_grating,
    margin=0.2
).save( dotenv['ATTACHMENT_DIR'] / 'moire_kategorien' / 'scale_1d.jpg' )

In [28]:
percentage_per_period = wavelength / grating_2d.shape[0]

for period_factor in [0.3,0.5,1,5]:
    scale_factor = percentage_per_period * period_factor
    scale_by_level = lambda img: img.resize((round(img.size[0] * (1 + scale_factor)), round(img.size[1] * (1 + scale_factor))))
    filename = f'scale_{ str(period_factor).replace(".","") }'
    
    get_fft_spectrum( grating_img_2d, scale_by_level ).write_image( dotenv['ATTACHMENT_DIR'] / 'moire_kategorien' / f'{filename}_1d_fft.jpg' )
    get_effect_img(
        grating_img_2d,
        scale_by_level,
        margin=0.2
    ).save( dotenv['ATTACHMENT_DIR'] / 'moire_kategorien' / f'{filename}_1d.jpg' )

In [29]:
for scale_factor_x in [0.01,0.05,0.08,0.1]:
    scale_factor_y = 0.05
    scale_by_level = lambda img: img.resize((round(img.size[0] * (1 + scale_factor_x)), round(img.size[1] * (1 + scale_factor_y))))
    filename = f'scale_{ str(scale_factor_x).replace("0.","") }_{ str(scale_factor_y).replace("0.","") }'
    
    get_fft_spectrum( grating_img_2d, scale_by_level ).write_image( dotenv['ATTACHMENT_DIR'] / 'moire_kategorien' / f'{filename}_1d_fft.jpg' )
    get_effect_img(
        grating_img_2d,
        scale_by_level,
        margin=0.2
    ).save( dotenv['ATTACHMENT_DIR'] / 'moire_kategorien' / f'{filename}_1d.jpg' )

In [30]:
for scale_factor_x in [0.01,0.05,0.08,0.1]:
    scale_factor_y = scale_factor_x
    scale_by_level = lambda img: img.resize((round(img.size[0] * (1 + scale_factor_x)), round(img.size[1] * (1 + scale_factor_y))))
    filename = f'scale_{ str(scale_factor_x).replace("0.","") }_{ str(scale_factor_y).replace("0.","") }'
    
    get_fft_spectrum( grating_img_2d, scale_by_level ).write_image( dotenv['ATTACHMENT_DIR'] / 'moire_kategorien' / f'{filename}_1d_fft.jpg' )
    get_effect_img(
        grating_img_2d,
        scale_by_level,
        margin=0.2
    ).save( dotenv['ATTACHMENT_DIR'] / 'moire_kategorien' / f'{filename}_1d.jpg' )

In [19]:
t1 = 1
results = []

for i in range(100):
    t2 = 1 + ((i+1) / 100)
    
    results.append((t2, (t1 * t2) / abs(t1 - t2)))

In [20]:
px.line(
    x=[r[0] for r in results],
    y=[r[1] for r in results],
    labels={
        'x' : 'Skalierungsfaktor',
        'y' : 'Periode des Moirémusters'
    },
    color_discrete_sequence=COLORS['COLOR_SEQUENCE'],
    width=1000,
    height=350
).write_image( dotenv['ATTACHMENT_DIR'] / 'moire_kategorien' / f'abhaengigkeit_skalierungsfaktor_moire_periode.jpg' )

In [ ]:
scale_factor = 0.05
scale_grating = lambda img: img.resize((round(img.size[0] * (1 + scale_factor)), round(img.size[1] * (1 + scale_factor))))

get_fft_spectrum( grating_img_2d, scale_grating ).write_image( dotenv['ATTACHMENT_DIR'] / 'moire_kategorien' / 'scale_2d_fft.jpg' )
get_effect_img(
    grating_img_2d,
    scale_grating,
    margin=0.2
).save( dotenv['ATTACHMENT_DIR'] / 'moire_kategorien' / 'scale_2d.jpg' )

get_fft_spectrum( grating_img, scale_grating ).write_image( dotenv['ATTACHMENT_DIR'] / 'moire_kategorien' / 'scale_1d_fft.jpg' )
get_effect_img(
    grating_img,
    scale_grating,
    margin=0.2
).save( dotenv['ATTACHMENT_DIR'] / 'moire_kategorien' / 'scale_1d.jpg' )

# Blow Up

In [ ]:
get_effect_img(
    grating_img_2d,
    lambda img: contract(img, 8),
    margin=0.2
)

In [ ]:
blow_up(grating_img_2d, 8).save( dotenv['ATTACHMENT_DIR'] / 'moire_kategorien' / 'blow_up_base.jpg' )
contract(grating_img_2d, 8).save( dotenv['ATTACHMENT_DIR'] / 'moire_kategorien' / 'contract_base.jpg' )

get_fft_spectrum( grating_img_2d, lambda img: blow_up(img, 8), overlay=False ).write_image( dotenv['ATTACHMENT_DIR'] / 'moire_kategorien' / 'blow_up_base_fft.jpg' )
get_fft_spectrum( grating_img_2d, lambda img: contract(img, 8), overlay=False ).write_image( dotenv['ATTACHMENT_DIR'] / 'moire_kategorien' / 'contract_base_fft.jpg' )

In [ ]:
get_effect_img(
    grating_img_2d,
    lambda img: blow_up(img, 8),
    margin=0.2
).save( dotenv['ATTACHMENT_DIR'] / 'moire_kategorien' / 'blow_up_overlay.jpg' )
get_fft_spectrum( grating_img_2d, lambda img: blow_up(img, 8) ).write_image( dotenv['ATTACHMENT_DIR'] / 'moire_kategorien' / 'blow_up_overlay_fft.jpg' )

get_effect_img(
    grating_img_2d,
    lambda img: contract(img, 8),
    margin=0.2
).save( dotenv['ATTACHMENT_DIR'] / 'moire_kategorien' / 'contract_overlay.jpg' )
get_fft_spectrum( grating_img_2d, lambda img: contract(img, 8) ).write_image( dotenv['ATTACHMENT_DIR'] / 'moire_kategorien' / 'contract_overlay_fft.jpg' )

# wellenförmige Verzerrung

In [ ]:
wave_configurations = [
    (500,10),
    (300,15),
    (100,20),
    (50,5)
]

In [ ]:
for i in range(len(wave_configurations)):
    wave_deform_func = lambda img: wave_deform( img, wave_configurations[:i+1] )

    wave_deform_func( grating_img ).save(
        dotenv['ATTACHMENT_DIR'] / 'moire_kategorien' / f'wave_base_{ (i+1) }.jpg'
    )
    get_fft_spectrum( grating_img_2d, wave_deform_func, overlay=False ).write_image(
        dotenv['ATTACHMENT_DIR'] / 'moire_kategorien' / f'wave_base_{ (i+1) }_fft.jpg'
    )

    get_effect_img(
        grating_img_2d,
        wave_deform_func,
        margin=0.2
    ).save(
        dotenv['ATTACHMENT_DIR'] / 'moire_kategorien' / f'wave_overlay_{ (i+1) }.jpg'
    )
    get_fft_spectrum( grating_img_2d, wave_deform_func, overlay=True ).write_image(
        dotenv['ATTACHMENT_DIR'] / 'moire_kategorien' / f'wave_overlay_{ (i+1) }_fft.jpg'
    )

In [ ]:
from scipy.signal import fftconvolve

In [ ]:
fftconvolve([5,5],[3,3])

In [ ]:
x = np.arange(-1000, 1001, 1)
X, Y = np.meshgrid(x, x)
wavelength = 20
grating = np.sin(2 * np.pi * X / wavelength) * 0.5 + 0.5
grating[grating < 0] = 0
grating[grating > 1] = 1

grating_rotated = np.sin(
    2*np.pi*(X*np.cos(angle) + Y*np.sin(angle)) / wavelength
) * 0.5 + 0.77
grating_rotated[grating_rotated < 0] = 0
grating_rotated[grating_rotated > 1] = 1

In [ ]:
convolved = fftconvolve(
    get_fft(grating),
    get_fft(grating_rotated)
)

In [ ]:
px.imshow(limit_frequencies(get_fft(grating).real, inner_limit=10))

In [ ]:
px.imshow(limit_frequencies(get_fft(grating).real, inner_limit=10))